# The script cleans up and normalizes the addresses and contact information

##1. Load and clean up data

In [98]:
import pandas as pd
import numpy as np

def normalizeStreetName(row, addr_lookup):
    в = process.extractOne(row['Адрес'], choices=addr_lookup, score_cutoff=70)
    if abbrev:
        return state_to_code[abbrev[0]]
    return 'error'

def normalizeStreetType(row):
     
    if abbrev:
        return state_to_code[abbrev[0]]
    return 'error'


raw_data = pd.read_excel('sample.xlsx', sheetname='DB', header=2, encoding='utf-8').reset_index(drop=True)
raw_data.Адрес = raw_data.Адрес.replace(to_replace='(П|п)(Р|р)\.', value='просп. ', regex=True)


addr_lookup = pd.read_excel('sample.xlsx', sheetname='cat', header=1, encoding='utf-8')
addr_lookup = addr_lookup.reset_index(drop=True).dropna(axis=1, how='all')

raw_data.dropna(axis=0, how='all', inplace=True)
raw_data.drop(raw_data.columns[4:], axis=1, inplace=True)
# raw_data.dropna(axis=1, how='all', inplace=True)

#swap address and tel if misplaced
swap_idx = raw_data.Адрес.str.contains(r'^[\d\s-]+$', na=True) #index for rows to swap address and tel 
raw_data.loc[swap_idx, ['Адрес', 'Телефон 1']] = raw_data[swap_idx][['Телефон 1','Адрес']].values
raw_data.Адрес.fillna(value='<no address>', inplace=True)
raw_data[raw_data.Адрес.str.contains('<no address>')]

ukr_addr = pd.read_table('streets_ukr_for_data_preprocessed.txt', header=None, names=['Адрес'])
rus_addr = pd.read_table('streets_rus_for_data_preprocessed.txt', header=None, names=['Адрес'])
raw_data['rus_addr'] = rus_addr
raw_data.rus_addr.replace(to_replace='(П|п)роспект', value='просп.', regex=True, inplace=True)
raw_data.rus_addr.replace(to_replace='(П|п)-т', value='просп. ', regex=True, inplace=True)
raw_data.rus_addr.replace(to_replace='^(В|в)ул(\.|\s)', value='ул. ', regex=True, inplace=True)
raw_data.rus_addr.replace(to_replace='^(В|в)(\.|\s)', value='ул. ', regex=True, inplace=True)
raw_data.rus_addr.replace(to_replace='^(Н|н)(\.|\s)?(аб)?.*Перемог.', value='ул. Набережная Победы', regex=True, inplace=True)
# raw_data.Адрес[~raw_data.Адрес.isin(ukr_addr.Адрес)]
# len(ukr_addr), len(raw_data), len(rus_addr)
raw_data.head()

,Качество,Адрес,Телефон 1,Телефон 2,rus_addr
0,плохое,"вул. Дємєнтьєва, 2/71",066-574-01-31,NaN,"ул. Дементьева, 2/71"
1,плохое,"Вул.Железнодорожна,15",0,NaN,"ул.Железнодорожна, 15"
2,плохое,"ул.Гагарина, 37 кв.4 (Першотравенск)",502240878,NaN,"ул.Гагарина, 37 кв.4 (Першотравенск)"
3,плохое,"просп. Гагарина, д37 кв.4 Першотравенск",502240878,NaN,"просп. Гагарина, Д37 кв.4 Першотравенск"
4,плохое,Наб. Перемоги 130/1/189,675688508,NaN,Наб. Победы 130/1/189


## Split street name and address

In [99]:
split_topol_addr_regex = r'^(\D*|.+\d)(?=(?<=Тополь 1|Т-1|Т-2|Т-3|Т-4)\D*\d)(.*)$'
split_topol_addr_regex_named = r'^(?P<Street>\D*|.+\d)(?=(?<=Тополь 1|Т-1|Т-2|Т-3|Т-4)\D*\d)(?P<addr>.*)$'

split_addr_regex = r'^(\D*|.+\d)(?=\D*\d)(.*)$'
split_addr_regex_named=r'^(?P<street>\D*)(?P<addr>.*)$'

raw_data['street'], raw_data['addr'] = zip(*raw_data.rus_addr.str.extract(split_addr_regex_named).values)
raw_data.head(10)


,Качество,Адрес,Телефон 1,Телефон 2,rus_addr,street,addr
0,плохое,"вул. Дємєнтьєва, 2/71",066-574-01-31,NaN,"ул. Дементьева, 2/71","ул. Дементьева,",2/71
1,плохое,"Вул.Железнодорожна,15",0,NaN,"ул.Железнодорожна, 15","ул.Железнодорожна,",15
2,плохое,"ул.Гагарина, 37 кв.4 (Першотравенск)",502240878,NaN,"ул.Гагарина, 37 кв.4 (Першотравенск)","ул.Гагарина,",37 кв.4 (Першотравенск)
3,плохое,"просп. Гагарина, д37 кв.4 Першотравенск",502240878,NaN,"просп. Гагарина, Д37 кв.4 Першотравенск","просп. Гагарина, Д",37 кв.4 Першотравенск
4,плохое,Наб. Перемоги 130/1/189,675688508,NaN,Наб. Победы 130/1/189,Наб. Победы,130/1/189
5,плохое,в.Красночечелівська//б.46 кв.22,500514364,NaN,ул. Красночечеливська // б.46 кв.22,ул. Красночечеливська // б.,46 кв.22
6,плохое,"п-т Ілліча, буд. 21, кв. 75",0960414961 0960414963,NaN,"просп. Ильича, д. 21 кв. 75","просп. Ильича, д.",21 кв. 75
7,плохое,ул.Володарского 109а,501926152,NaN,ул.Володарского 109а,ул.Володарского,109а
8,плохое,"Запорізьке шосе, б.40, кв.357",994695916; 0666850000,NaN,"Запорожское шоссе, б.40, кв.357","Запорожское шоссе, б.","40, кв.357"
9,плохое,"просп. Миру, 71/62",097-361-00-41,NaN,"просп. Мира, 71/62","просп. Мира,",71/62


## Translate street names (don't abuse, might be Google-banned )

In [ ]:
import goslate
from urllib import request


proxy_handler = request.ProxyHandler({"http" : "http://176.31.119.64:8080"})
proxy_opener = request.build_opener(request.HTTPHandler(proxy_handler), request.HTTPSHandler(proxy_handler))
gs = goslate.Goslate(opener=proxy_opener)
streets = [str(x) for x in raw_data.Адрес.tolist()]
streets_rus = gs.translate(streets, 'ru', source_language='uk')
translation = list(streets_rus)

In [3]:
from pyspark import  SparkContext, SQLContext
sc = SparkContext( 'local', 'pyspark')



## String matching heuristics

##Lookup and match street names

In [112]:
# %debug
import ipdb
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time


street_choices = [str(x) for x in addr_lookup.Улица.dropna().tolist()]

def match_scores(st):
    return [(fuzz.token_set_ratio(st, normalized_st), normalized_st, st) for normalized_st in street_choices]

def reducer(score1, score2):  
    print(score1, score2)    
    return max(score1, score2, key=lambda x: x[0])
#     scores = [(fuzz.token_set_ratio(raw, normal), normal) for normal in street_choices]
#     return max(scores, key=lambda x: x[0])[1]


start = time.clock()

dirty_streets = sc.parallelize(raw_data.street.head(10).str.encode(encoding='utf-8').tolist())
# r = dirty_streets.flatMap(match_scores).reduceByKey(reducer).collect()

# for (x,y) in zip(r, raw_data.street.head(10)):
#     print (x,y)

print('time: ' + str(time.clock() - start))



start = time.clock()
# streets_dictionary = prepare_match_list(street_choices)

G1 = ngram.NGram(street_choices, N=1)
G2 = ngram.NGram(street_choices, N=2)
G3 = ngram.NGram(street_choices, N=3)
G4 = ngram.NGram(street_choices, N=4)
G = G2

matched_streets = list()
# for raw in raw_data.street.tolist():    
#     scores = [(fuzz.token_set_ratio(raw, normal), normal) for normal in G.search(raw, threshold=0.3)]
#     if (scores):
#         matched_streets.append(max(scores, key=lambda x: x[0])[1])
#     match = process.extractOne(raw, street_choices)
#     matched_streets.append(match[0])
    
# print(matched_streets)
print('time ngram on test data: ' + str(time.clock() - start))


# pd.isnull(addr_lookup['Unnamed: 1'])
# addr_lookup['Unnamed: 1'].iloc[1]

# df = raw_data.dropna(how='all')
# # df = raw_data.drop(raw_data.index[1])
# df = df[df.Адрес.str.contains(u'Правд')==True]
# df



choices = ['просп. Воронцова', 'просп. Гагарина', 'просп. Газеты "Правда"', 'просп. Героев', 'просп. Ильича', 
           'просп. Карла Маркса', 'просп. Кирова', 'просп. Металлургов', 'просп. Мира', 'просп. Олимпийский',
           'просп. Петровского', 'просп. Пушкина', 'просп. Свободы', 'просп. Сергея Нигояна', 'просп. Труда',
           'ул. Правды', 'ул. Юдина', 'ул. Набережная',  'ул. Набережная В.И.Ленина', 'пер. Пролетарской Победы', 
           'ул. Набережная Заводская', 'ул. Набережная им. Ленина', 'ул. Набережная Победы', 
           'ул. Мира', 'ул. Железнодорожная', 'ул. Дорожная', 'пер. Пролетарской Победы']    

# process.extract(u'ул. Железнодорожна', choices)
# fuzz.partial_token_sort_ratio(u'Наб. Победы', 'ул. Набережная Победы')
# fuzz.partial_ratio(u'Наб. Победы', 'ул. Набережная Победы')


# sorted([(fuzz.QRatio('ул.Малиновського,', normal), normal) for normal in street_choices], reverse=True)

start = time.clock()
[fuzz.token_set_ratio('ул.Малиновського,', normal) for normal in street_choices]
print('time: ' + str(time.clock() - start))

import matplotlib.pyplot as plt
# plt.plot([fuzz.token_set_ratio('ул.Малиновського,', normal) for normal in street_choices])
# plt.show()
sorted(street_choices)
import ngram

G = ngram.NGram(street_choices, N=2)
start = time.clock()
[fuzz.token_set_ratio('ул.Малиновського,', normal) for normal in G.search('ул.Малиновського,', threshold=0.2)]
print('time: ' + str(time.clock() - start))



time: 0.004289191580028273
time ngram on test data: 0.1970210003637476
time: 0.3774488589842804
time: 0.04786831882665865


In [115]:
start = time.clock()
# test = raw_data.street.tolist()
# matches = [(x, G.finditem(x)) for x in test]
# print('time: ' + str(time.clock() - start))

G2.searchitem('просп. Правды', )

# for x in matches:
#     print(x)


[('ул. Правды', 0.47058823529411764),
 ('просп. Свободы', 0.45),
 ('просп. Мира', 0.4444444444444444),
 ('просп. Газеты "Правда"', 0.4230769230769231),
 ('просп. Пушкина', 0.38095238095238093),
 ('просп. Труда', 0.35),
 ('просп. Кирова', 0.3333333333333333),
 ('просп. Героев', 0.3333333333333333),
 ('просп. Ильича', 0.3333333333333333),
 ('просп. Петровского', 0.32),
 ('просп. Гагарина', 0.30434782608695654),
 ('просп. Воронцова', 0.2916666666666667),
 ('просп. Олимпийский', 0.2692307692307692),
 ('просп. Металлургов', 0.2692307692307692),
 ('просп. Карла Маркса', 0.25925925925925924),
 ('пер. Просторный', 0.25),
 ('просп. Сергея Нигояна', 0.2413793103448276),
 ('пер. Протавчанский', 0.2222222222222222),
 ('пер. Проскуровский', 0.2222222222222222),
 ('пер. Пролетарской Победы', 0.21875),
 ('пер. Проектный', 0.20833333333333334),
 ('ул. Просторная', 0.20833333333333334),
 ('пер. Проездной', 0.20833333333333334),
 ('пер. Прохладный', 0.2),
 ('пер. Пригородный', 0.19230769230769232),
 ('п

[2, 3]

In [ ]:
from fuzzywuzzy import utils, fuzz

@utils.check_for_none
def _token_set(s1, dict_entry, partial=True, force_ascii=True):
    """Find all alphanumeric tokens in each string...
        - treat them as a set
        - construct two strings of the form:
            <sorted_intersection><sorted_remainder>
        - take ratios of those two strings
        - controls for unordered partial matches
        
        Note: dict_entry: a pair where 1st - string, 2nd - tokens """

    p1 = utils.full_process(s1, force_ascii=force_ascii)
    p2 = dict_entry[0]

    if not utils.validate_string(p1):
        return 0
    if not utils.validate_string(p2):
        return 0

    # pull tokens
    tokens1 = set(utils.full_process(p1).split())
    tokens2 = dict_entry[1]

    intersection = tokens1.intersection(tokens2)
    diff1to2 = tokens1.difference(tokens2)
    diff2to1 = tokens2.difference(tokens1)

    sorted_sect = " ".join(sorted(intersection))
    sorted_1to2 = " ".join(sorted(diff1to2))
    sorted_2to1 = " ".join(sorted(diff2to1))

    combined_1to2 = sorted_sect + " " + sorted_1to2
    combined_2to1 = sorted_sect + " " + sorted_2to1

    # strip
    sorted_sect = sorted_sect.strip()
    combined_1to2 = combined_1to2.strip()
    combined_2to1 = combined_2to1.strip()

    if partial:
        ratio_func = fuzz.partial_ratio
    else:
        ratio_func = fuzz.ratio

    pairwise = [
        ratio_func(sorted_sect, combined_1to2),
        ratio_func(sorted_sect, combined_2to1),
        ratio_func(combined_1to2, combined_2to1)
    ]
    return max(pairwise)

def token_set_ratio(s1, dictionary, force_ascii=True):
    return _token_set(s1, dictionary, partial=False, force_ascii=force_ascii)

def prepare_match_list(choices):
    choices_processed = list()
    for choice in choices:
        p2 = utils.full_process(choice, force_ascii=True)
        tokens2 = set(utils.full_process(p2).split())
        choices_processed.append( (p2, tokens2) )
    return choices_processed
    
def match(s1, choices):
    return [(token_set_ratio(s1, c), c[0]) for c in choices]
    
    
street_choices = [str(x) for x in addr_lookup.Улица.dropna().tolist()]
streets_dictionary = prepare_match_list(street_choices)    
match(raw, streets_dictionary)

In [ ]:
translation

In [314]:
import heapq
choices = addr_lookup.Улица.tolist()
choices = [str(x) for x in choices]              
# choices = process.extract("проспект Правди", choices, limit=2)
# print(choices)    
# choices = ['просп. Воронцова', 'просп. Гагарина', 'просп. Газеты "Правда"', 'просп. Героев', 'просп. Ильича', 
#            'просп. Карла Маркса', 'просп. Кирова', 'просп. Металлургов', 'просп. Мира', 'просп. Олимпийский',
#            'просп. Петровского', 'просп. Пушкина', 'просп. Свободы', 'просп. Сергея Нигояна', 'просп. Труда',
#            'ул. Правды']    
process.extract(u'Т-1', choices)
# items = [(fuzz.partial_token_sort_ratio('проспект Правди', w), w) for w in choices]
# heapq.nlargest(3, items, key=lambda item: item[0])

    

[('ж/м Тополь-1', 85),
 ('пер. 1-й Наклонный', 85),
 ('пер. Веснина 1-й', 85),
 ('тупик Одоевского 1-й', 85),
 ('тупик Рыбинский 1-й', 85)]

In [8]:
from numbapro import vectorize, guvectorize
import numpy as np
from numbapro import cuda
import fuzzywuzzy

@cuda.reduce
def score(s1, s2):
    return fuzz.token_set_ratio(s1, s2)

score()
np.string_

SyntaxError: invalid syntax (<ipython-input-8-7821a9875f95>, line 10)